In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Load data
data_path = 'sum_sum.csv'
data = pd.read_csv(data_path)

# Extract features and target variables
X = data[['UTS', 'YS', 'EL', 'Temperature']]
y_compound_1 = data['Si']
y_compound_2 = data['Cu']
y_compound_3 = data['Mn']
y_compound_4 = data['Ni']
y_compound_5 = data['Fe']
y_compound_9 = data['Mg']
y_compound_13 = data['Ti']

X_train_1, X_test_1, y_train_compound_1, y_test_compound_1 = train_test_split(X, y_compound_1, test_size=0.2, random_state=42)
X_train_2, X_test_2, y_train_compound_2, y_test_compound_2 = train_test_split(X, y_compound_2, test_size=0.2, random_state=42)
X_train_3, X_test_3, y_train_compound_3, y_test_compound_3 = train_test_split(X, y_compound_3, test_size=0.2, random_state=42)
X_train_4, X_test_4, y_train_compound_4, y_test_compound_4 = train_test_split(X, y_compound_4, test_size=0.2, random_state=42)
X_train_5, X_test_5, y_train_compound_5, y_test_compound_5 = train_test_split(X, y_compound_5, test_size=0.2, random_state=42)
X_train_9, X_test_9, y_train_compound_9, y_test_compound_9 = train_test_split(X, y_compound_9, test_size=0.2, random_state=42)
X_train_13, X_test_13, y_train_compound_13, y_test_compound_13 = train_test_split(X, y_compound_13, test_size=0.2, random_state=42)

pt = PowerTransformer(method='yeo-johnson', standardize=True)
X_train_transformed = pt.fit_transform(X_train_1)
X_test_transformed = pt.transform(X_test_1)

poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train_transformed)
X_test_poly = poly.transform(X_test_transformed)

ada_feature_selector = AdaBoostRegressor(n_estimators=100, random_state=42)
selector = SelectFromModel(ada_feature_selector, threshold='median')
X_train_selected = selector.fit_transform(X_train_poly, y_train_compound_1)  # 选择其中一个合金成分的训练集
X_test_selected = selector.transform(X_test_poly)

base_model = AdaBoostRegressor(n_estimators=100, random_state=42)

model_compound_1 = RandomForestRegressor(n_estimators=100, random_state=42)
model_compound_2 = RandomForestRegressor(n_estimators=50, random_state=200)
model_compound_3 = RandomForestRegressor(n_estimators=100, random_state=42)
model_compound_4 = RandomForestRegressor(n_estimators=100, random_state=42)
model_compound_5 = RandomForestRegressor(n_estimators=100, random_state=42)
model_compound_9 = RandomForestRegressor(n_estimators=100, random_state=42)
model_compound_13 = RandomForestRegressor(n_estimators=100, random_state=42)

model_compound_1.fit(X_train_selected, y_train_compound_1)
model_compound_2.fit(X_train_selected, y_train_compound_2)
model_compound_3.fit(X_train_selected, y_train_compound_3)
model_compound_4.fit(X_train_selected, y_train_compound_4)
model_compound_5.fit(X_train_selected, y_train_compound_5)
model_compound_9.fit(X_train_selected, y_train_compound_9)
model_compound_13.fit(X_train_selected, y_train_compound_13)

# 获取回归模型的预测结果作为新特征
predictions_compound_1 = model_compound_1.predict(X_test_selected)
predictions_compound_2 = model_compound_2.predict(X_test_selected)
predictions_compound_3 = model_compound_3.predict(X_test_selected)
predictions_compound_4 = model_compound_4.predict(X_test_selected)
predictions_compound_5 = model_compound_5.predict(X_test_selected)
predictions_compound_9 = model_compound_9.predict(X_test_selected)
predictions_compound_13 = model_compound_13.predict(X_test_selected)

# 构建新特征矩阵
new_features = np.column_stack((predictions_compound_1, predictions_compound_2, predictions_compound_3,
                                predictions_compound_4, predictions_compound_5, predictions_compound_9,
                                predictions_compound_13))

# 使用AdaBoost集成模型进行预测和评估
final_model = AdaBoostRegressor(base_model, n_estimators=100, random_state=42)

# 计算并输出每个合金成分的最终R²评分
compounds = [y_compound_1, y_compound_2, y_compound_3, y_compound_4, y_compound_5, y_compound_9, y_compound_13]

compound_names = ['Si', 'Cu', 'Mn', 'Ni', 'Fe','Mg','Ti']

save_path = r'D:\\XiaoHao_Research\\Machine_Learning_Heat_Resistant_Aluminum_Alloy\\'

for y_train_compound, y_test_compound, compound_name in zip(
    [y_train_compound_1, y_train_compound_2, y_train_compound_3, 
     y_train_compound_4, y_train_compound_5, y_train_compound_9,
     y_train_compound_13], 
    [y_test_compound_1, y_test_compound_2, y_test_compound_3, 
     y_test_compound_4, y_test_compound_5, y_test_compound_9,
     y_test_compound_13], compound_names):

    
    final_model.fit(new_features, y_test_compound)
    final_predictions = final_model.predict(new_features)
    r2_final_test = r2_score(y_test_compound, final_predictions)
    final_mse = mean_squared_error(y_test_compound, final_predictions)
    print(f"Final R² for {compound_name}: {r2_final_test}")
    print(f"Final MSE: {final_mse}")
    
    plt.figure(figsize=(15, 10))
    plt.scatter(y_test_compound, final_predictions, color='blue')
    plt.title(f'AdaBoost--{compound_name} ', fontsize=36, fontname="Times New Roman",fontweight='bold')
    plt.xlabel("True", fontsize=36, fontname="Times New Roman", fontweight='bold')
    plt.ylabel("Predict", fontsize=36, fontname="Times New Roman", fontweight='bold')
    
    plt.text(0.05, 0.95, f'R² = {r2_final_test:.2f}', fontsize=36,
             color='black', fontname="Times New Roman", fontweight='bold', transform=plt.gca().transAxes,
             verticalalignment='top')
    plt.text(0.05, 0.9, f'MSE = {final_mse:.2f}', fontsize=36,
             color='black', fontname="Times New Roman", fontweight='bold', transform=plt.gca().transAxes,
             verticalalignment='top')

    plt.text(0.05, 0.85, '-----', fontsize=36,
             color='red', fontname="Times New Roman", fontweight='bold', transform=plt.gca().transAxes,
             verticalalignment='top')
    plt.text(0.12, 0.85, ': True value = Predict value', fontsize=36,
             color='black', fontname="Times New Roman", fontweight='bold', transform=plt.gca().transAxes,
         verticalalignment='top')
    plt.xticks(fontsize=36, fontname="Times New Roman", fontweight='bold')
    plt.yticks(fontsize=36, fontname="Times New Roman", fontweight='bold')
    plt.gca().spines['top'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['right'].set_linewidth(2)
    plt.plot([y_test_compound.min(), y_test_compound.max()], [y_test_compound.min(), y_test_compound.max()], 'r--', lw=2)
    
    plt.savefig(f'{save_path}{compound_name}.png', dpi=300, bbox_inches='tight')
    plt.close() 